In [137]:
import pandas as pd
import numpy as np
# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

In [138]:
# Import and one more rename
data = pd.read_csv("../data/UsdanRegistration2016-2018.csv", encoding="utf-8")

data = data.rename({'reg_STDT::student_serialnumber_t':'student_id', 'reg_stdt_PRNT::add_bill_county_t' : 'student_county'}, axis=1)
data.groupby("student_id").nunique()["year"]

# filter by music
data = data.query("dept_t=='Chorus' or dept_t=='Orchestra' or dept_t=='Band'")
data.query("year!=2018").index.values

array([1682, 1687, 1688, 1690, 1692, 1693, 1694, 1702, 1705, 1714, 1717,
       1725, 1730, 1738, 1742, 1748, 1751, 1773, 1775, 1796, 1801, 1803,
       1807, 1808, 1820, 1829, 1830, 1837, 1839, 1840, 1847, 1854, 1858,
       1870, 1875, 1876, 1877, 1879, 1888, 1893, 1894, 1898, 1902, 1916,
       1922, 1924, 1925, 1926, 1939, 1940, 1941, 1953, 1954, 1959, 1966,
       1967, 1968, 1972, 1985, 1987, 1988, 1989, 1990, 1994, 2004, 2007,
       2009, 2017, 2018, 2019, 2026, 2027, 2034, 2035, 2040, 2041, 2044,
       2048, 2051, 2052, 2064, 2065, 2066, 2069, 2072, 2080, 2093, 2110,
       2111, 2137, 2143, 2146, 2157, 2164, 2170, 2187, 2204, 2205, 2214,
       2219, 2225, 2232, 2237, 2244, 2247, 2250, 2272, 2277, 2288, 2307,
       2338, 2358, 2365, 2367, 2369, 2384, 2388, 2389, 2401, 2402, 2403,
       2412, 2415, 2416, 2426, 2427, 2448, 2454, 2463, 2482, 2486, 2494,
       2506, 2508, 2515, 2516, 2531, 2541, 2542, 2543, 2547, 2555, 2568,
       2569, 2570, 2579, 2586, 2587, 2588, 2592, 25

In [139]:
# Make the data categorical
data["Major"] = data["Major"].astype('category')
data["Minor1"] = data["Minor1"].astype('category')
data["Minor2"] = data["Minor2"].astype('category')
data["dept_t"] = data["dept_t"].astype('category')
data["student_county"] = data["student_county"].astype('category')
data["program_type"] = data["reg_PROGRAMS::program_t"].astype('category')
data["LargeAid"] = data["_TuitionAidTotal"]
data["LargeAid"] = data["LargeAid"].fillna(data["_ScholarshipTotal"])

In [140]:
# Make Numerical Categorical Info and uniq classes
cat_columns = data.select_dtypes(['category']).columns
for cat in cat_columns:
    data[cat+"_str"] = cat + ": " + data[cat].astype('unicode')
data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)

In [141]:
# Separate Scholarship and Full-Paying Students
non_schol = data[pd.isnull(data["LargeAid"])]
schol = data.dropna(axis=0, subset=["LargeAid"])
print(non_schol.shape)
print(schol.shape)

(424, 33)
(517, 33)


In [142]:
# Check if the student returned between 2017 and 2018
# This can be improved to give more data
# df = data
df = non_schol # look at just non-scholarship students
print(df.shape)
recurrence = ((df[df["year"] != 2016].groupby("student_id").year.nunique() > 1)).to_frame()
recurrence = recurrence.rename({'year': 'returned'}, axis=1)
recurrence = non_schol[non_schol["year"] != 2016].join(recurrence, on="student_id")
recurrence = recurrence[recurrence["year"] == 2017]

(424, 33)


In [143]:
# Turn Categorical Categories Into Boolean because it makes the vectors more logical mathematically
return_inter = recurrence[["Major_str", "Minor1_str", "dept_t_str", "Minor2_str", "program_type_str", "student_county_str", "returned"]]
major = pd.get_dummies(return_inter.Major_str)
minor1 = pd.get_dummies(return_inter.Minor1_str)
minor2 = pd.get_dummies(return_inter.Minor2_str)
minor2 = pd.get_dummies(return_inter.Minor2_str)
dept = pd.get_dummies(return_inter.dept_t_str)
county = pd.get_dummies(return_inter.student_county_str)
program_type = pd.get_dummies(return_inter.program_type_str)
returned = return_inter["returned"].to_frame()
return_preds = returned.join([dept, program_type, major, county])

In [144]:
improvement = []
overfit = []
coefficients = []
# Janky K-Folds Cross Validation
# Randomly Picks Chunks of Data
# Then averages across all to account for variance
for i in range(10):
    msk = np.random.rand(len(return_preds)) < 0.8
    train = return_preds[msk]
    test = return_preds[~msk]
    y_train = train['returned'].values
    y_test = test['returned'].values
    X_train = train.drop(['returned'], axis=1).values
    X_test = test.drop(['returned'], axis=1).values

    # Naive
    naive_pred = [True]*len(y_test)
    naive_accuracy = round(metrics.accuracy_score(y_test, naive_pred) * 100, 2)
    naive_accuracy

    methods = [LogisticRegression(), LinearSVC(), SVC(), Perceptron()]
    for method in methods:
        improvement.append([])
        overfit.append([])
        coefficients.append([])
    for i, method in enumerate(methods):
        method.fit(X_train, y_train)
        if i in [0,1]:
            coeff_df = pd.DataFrame(train.columns.delete(train.columns.get_loc("returned")))
            coeff_df.columns = ['Feature']
            coeff_df["Correlation"] = pd.Series(method.coef_[0])
            coefficients[i].append(coeff_df.set_index("Feature"))
        Y_pred = method.predict(X_test)
        method_accuracy = round(metrics.accuracy_score(y_test, Y_pred) * 100, 2)
        acc_method = round(method.score(X_train, y_train) * 100, 2)
        improvement[i].append(method_accuracy)
        overfit[i].append(acc_method-method_accuracy)

C:\Users\Apache\Miniconda3\envs\dash\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.perceptron.Perceptron'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [145]:
# Print out the meaningful metrics where it counts
for i, method in enumerate(methods):
    np_improvement = np.array(improvement[i])
    np_overfit = np.array(overfit[i])
    print("\n"+str(method.__class__).split(".")[-1].split("'")[0])
    print("\tAverage Improvement:\t" + str(np.mean(np_improvement, axis=0)))
    print("\tAverage Overfit:\t" + str(np.mean(np_overfit, axis=0)))
    print("\tSTD Improvement:\t" + str(np.std(np_improvement, axis=0)))
    print("\tSTD Overfit:\t\t" + str(np.std(np_overfit, axis=0)))
    if i in [0,1]:
        print(pd.concat(coefficients[i]).groupby(pd.concat(coefficients[i]).index).mean().sort_values(by="Correlation", ascending=False))


LogisticRegression
	Average Improvement:	67.543
	Average Overfit:	6.621999999999998
	STD Improvement:	6.894280310518279
	STD Overfit:		7.437894594574463
                             Correlation
Feature                                 
Major: Clarinet                 0.949355
student_county: Westchester     0.635892
Major: Jazz Ensemble            0.561901
Major: Violin                   0.548897
student_county: nan             0.366865
program_type: 7 Week            0.194409
Major: Trombone                 0.134315
Major: Viola                    0.127172
Major: Trumpet                  0.126062
Major: Senior Chorus            0.110262
Major: Flute                    0.109886
student_county: Queens          0.101784
Major: Tuba                     0.082188
dept_t: Band                    0.074422
program_type: 4 Week            0.070388
dept_t: Orchestra               0.031398
student_county: Manhattan      -0.005591
Major: Saxophone               -0.046272
Major: Baritone           

In [146]:
msk = np.random.rand(len(return_preds)) < 0.8
train = return_preds[msk]
test = return_preds[~msk]

In [147]:
minor1

,Minor1: Actor’s Craft,Minor1: Archery,Minor1: Chamber Music,Minor1: Chamber Music (clarinet),Minor1: Chamber Music (Cello),Minor1: Chamber Music (Clarinet),Minor1: Chamber Music (French Horn),Minor1: Chamber Music (Trombone),Minor1: Chamber Music (percussion),Minor1: Chess,...,"Minor1: Nature, Ecology & Design",Minor1: Organic Gardening,Minor1: Painting & Drawing,Minor1: Photography,Minor1: Pilobolus: Moving & Making,Minor1: Quidditch,Minor1: Senior Chorus,Minor1: Sustainable Art: Little Changes with Little Sun,Minor1: Tap Dance,Minor1: nan
1682,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1688,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1692,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1694,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1702,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1705,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1714,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1725,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1742,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
